In [1]:
from pmagpy import ipmag
import os
import json
import numpy as np


import pandas as pd
from pandas import DataFrame
from pmagpy import builder
from pmagpy import validate_upload
import sys

pmagpy_dir = os.path.join(os.getcwd(), 'pmagpy')
if pmagpy_dir not in sys.path:
    sys.path.append(pmagpy_dir)

# Object oriented way

In [2]:
reload(ipmag)
working_dir = "/Users/nebula/Python/PmagPy/3_0/"
fname = "contribution_11087_v3.0.txt"
working_dir = "/Users/nebula/Python/PmagPy/3_0"
#os.listdir("/Users/nebula/Desktop/3_0")
ipmag.download_magic(fname, working_dir, working_dir, True, True)

working on:  'contribution'
1  records written to file  /Users/nebula/Python/PmagPy/3_0/contribution.txt
contribution  data put in  /Users/nebula/Python/PmagPy/3_0/contribution.txt
working on:  'locations'
4  records written to file  /Users/nebula/Python/PmagPy/3_0/locations.txt
locations  data put in  /Users/nebula/Python/PmagPy/3_0/locations.txt
working on:  'sites'
90  records written to file  /Users/nebula/Python/PmagPy/3_0/sites.txt
sites  data put in  /Users/nebula/Python/PmagPy/3_0/sites.txt
location_1:  Osler Volcanics, Nipigon Strait, Lower Reversed
directory  /Users/nebula/Python/PmagPy/3_0/Location_1  already exists - overwriting everything: True
unpacking:  /Users/nebula/Python/PmagPy/3_0/contribution.txt
1  read in
unpacking:  /Users/nebula/Python/PmagPy/3_0/locations.txt
4  read in
2  records written to file  /Users/nebula/Python/PmagPy/3_0/Location_1/locations.txt
2  stored in  /Users/nebula/Python/PmagPy/3_0/Location_1/locations.txt
unpacking:  /Users/nebula/Python/Pmag

True

In [3]:
def get_data_model():
    model_file = os.path.join('3_0', 'MagIC Data Model v3.0 - unpublished.json')
    f = open(model_file, 'r')
    string = '\n'.join(f.readlines())
    raw = json.loads(unicode(string, errors='ignore'))
    full = DataFrame(raw)
    return full

In [4]:
reload(builder)
reload(validate_upload)
dmodel = get_data_model()
b = builder.ErMagicBuilder(working_dir, dmodel)
b.get_data()
b.get_all_magic_info()

-I- No magic_measurements.txt file
-I- No magic_measurements.txt file
-I- Getting specimen info
-I- Attempting to read /Users/nebula/Python/PmagPy/3_0/specimens.txt
-W- Could not find /Users/nebula/Python/PmagPy/3_0/specimens.txt
-I- Getting sample info
-I- Attempting to read /Users/nebula/Python/PmagPy/3_0/samples.txt
-W- Could not find /Users/nebula/Python/PmagPy/3_0/samples.txt
-I- Getting site info
-I- Attempting to read /Users/nebula/Python/PmagPy/3_0/sites.txt
-I- Getting location info
-I- Attempting to read /Users/nebula/Python/PmagPy/3_0/locations.txt
-W- Could not find /Users/nebula/Python/PmagPy/3_0/ages.txt


In [5]:
b.sites[:5]

[site: 24, site: 25, site: 26, site: 27, site: 20]

In [6]:
b.sites[0].data

{'bed_dip': '',
 'bed_dip_direction': '',
 'citations': 'Cox & Doell 1960:McElhinny & McFadden 2000',
 'conglomerate_test': 'ND',
 'description': 'VGP:Site 24',
 'dir_dec': 116.8,
 'dir_inc': '-40.4',
 'dir_k': '',
 'dir_n_samples': '1',
 'dir_polarity': '',
 'dir_tilt_correction': '100',
 'geologic_classes': '',
 'geologic_types': '',
 'lat': '48.678789',
 'lithologies': '',
 'location_name': 'Osler Volcanics, Nipigon Strait, Lower Reversed',
 'lon': '271.942681',
 'method_codes': 'DE-DI:FS-LOC-GOOGLE:LP-DC2',
 'result_type': 'i',
 'site_name': '24',
 'vgp_lat': '34.61096611',
 'vgp_lon': 178.2474147,
 'vgp_n_samples': '1'}

In [7]:

def set_key(key, value, lst):
    for i in b.sites:
        i.data[key] = value
        
set_key('new_key', 'new_value', b.sites)

In [8]:
b.sites[0].data['new_key']

'new_value'

In [9]:
b.add_site('new_site', None, data=b.sites[0].data)
b.sites[-5:]

[site: 16, site: 19, site: 18, site: 30, site: new_site]

# DataFrame way

In [10]:
# set up site file as pandas DataFrame
from pmagpy import pmag
site_data, dtype = pmag.magic_read(os.path.join(working_dir, 'sites.txt'))
site_df = DataFrame(site_data)
site_df.index = site_df['site_name']
# replace '' with np.nan, so you can use isnull(), notnull(), etc.
# can always switch back with DataFrame.fillna('')
site_df[site_df == ''] = np.nan
del site_df['site_name']


In [11]:
# DataFrame of all read in sites
site_df[:5]

bed_dip bed_dip_direction  \
site_name                             
1              10               135   
1              10               135   
1             NaN               NaN   
2              12               140   
2              12               140   

                                            citations conglomerate_test  \
site_name                                                                 
1                                          This study               NaN   
1                                          This study               NaN   
1          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
2                                          This study               NaN   
2                                          This study               NaN   

          description dir_dec dir_inc dir_k dir_n_samples dir_polarity  \
site_name                                                                
1                 NaN   293.1    34.5   517             5            n   
1                 NaN   289.8    43.6   517             5            n   
1          VGP:Site 1   289.8    43.6   NaN             1          NaN   
2                 NaN   290.6    31.9   243             5            n   
2                 NaN   285.7      42   243             5            n   

               ...      geologic_types        lat lithologies  \
site_name      ...                                              
1              ...           Lava Flow        NaN      Basalt   
1              ...           Lava Flow        NaN      Basalt   
1              ...                 NaN  48.626429         NaN   
2              ...           Lava Flow        NaN      Basalt   
2              ...           Lava Flow        NaN      Basalt   

                                           location_name         lon  \
site_name                                                              
1          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
1          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
1          Osler Volcanics, Nipigon Strait, Upper Normal  271.894142   
2          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
2          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   

                                         method_codes result_type  \
site_name                                                           
1                      FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
1                      FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
1                          DE-DI:FS-LOC-GOOGLE:LP-DC2           i   
2          DE-K:LP-DC2:FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
2                      FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   

               vgp_lat      vgp_lon vgp_n_samples  
site_name                                          
1                  NaN          NaN           NaN  
1                  NaN          NaN           NaN  
1          31.65155202  185.5745062             1  
2                  NaN          NaN           NaN  
2                  NaN          NaN           NaN  

[5 rows x 22 columns]

In [12]:
# all sites with site_name (index) of '1'
# will return a smaller DataFrame or a Series if there is only 1
site_df.ix['1']

bed_dip bed_dip_direction  \
site_name                             
1              10               135   
1              10               135   
1             NaN               NaN   

                                            citations conglomerate_test  \
site_name                                                                 
1                                          This study               NaN   
1                                          This study               NaN   
1          Cox & Doell 1960:McElhinny & McFadden 2000                ND   

          description dir_dec dir_inc dir_k dir_n_samples dir_polarity  \
site_name                                                                
1                 NaN   293.1    34.5   517             5            n   
1                 NaN   289.8    43.6   517             5            n   
1          VGP:Site 1   289.8    43.6   NaN             1          NaN   

               ...      geologic_types        lat lithologies  \
site_name      ...                                              
1              ...           Lava Flow        NaN      Basalt   
1              ...           Lava Flow        NaN      Basalt   
1              ...                 NaN  48.626429         NaN   

                                           location_name         lon  \
site_name                                                              
1          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
1          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
1          Osler Volcanics, Nipigon Strait, Upper Normal  271.894142   

                             method_codes result_type      vgp_lat  \
site_name                                                            
1          FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN          NaN   
1          FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN          NaN   
1              DE-DI:FS-LOC-GOOGLE:LP-DC2           i  31.65155202   

               vgp_lon vgp_n_samples  
site_name                             
1                  NaN           NaN  
1                  NaN           NaN  
1          185.5745062             1  

[3 rows x 22 columns]

In [13]:
# index by position, will always return a single record as Series
# in this case, get the second record
site_df.ix[1]

bed_dip                                                           10
bed_dip_direction                                                135
citations                                                 This study
conglomerate_test                                                NaN
description                                                      NaN
dir_dec                                                        289.8
dir_inc                                                         43.6
dir_k                                                            517
dir_n_samples                                                      5
dir_polarity                                                       n
dir_tilt_correction                                              100
geologic_classes                                           Extrusive
geologic_types                                             Lava Flow
lat                                                              NaN
lithologies                       

In [14]:
# return all sites with the description column filled in
cond = site_df['description'].notnull()
site_df[cond]

bed_dip bed_dip_direction  \
site_name                             
1             NaN               NaN   
2             NaN               NaN   
3             NaN               NaN   
4             NaN               NaN   
5             NaN               NaN   
6             NaN               NaN   
7             NaN               NaN   
8             NaN               NaN   
9             NaN               NaN   
10            NaN               NaN   
11            NaN               NaN   
12            NaN               NaN   
13            NaN               NaN   
14            NaN               NaN   
15            NaN               NaN   
16            NaN               NaN   
17            NaN               NaN   
18            NaN               NaN   
19            NaN               NaN   
20            NaN               NaN   
21            NaN               NaN   
22            NaN               NaN   
23            NaN               NaN   
24            NaN               NaN   
25            NaN               NaN   
26            NaN               NaN   
27            NaN               NaN   
28            NaN               NaN   
29            NaN               NaN   
30            NaN               NaN   

                                            citations conglomerate_test  \
site_name                                                                 
1          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
2          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
3          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
4          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
5          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
6          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
7          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
8          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
9          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
10         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
11         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
12         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
13         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
14         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
15         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
16         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
17         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
18         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
19         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
20         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
21         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
22         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
23         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
24         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
25         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
26         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
27         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
28         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
29         Cox & Doell 1960:McElhinny & McFadden 2000                ND   
30         Cox & Doell 1960:McElhinny & McFadden 2000                ND   

           description dir_dec dir_inc dir_k dir_n_samples dir_polarity  \
site_name                                                                 
1           VGP:Site 1   289.8    43.6   NaN             1          NaN   
2           VGP:Site 2   285.7      42   NaN             1          NaN   
3           VGP:Site 3   307.4    37.9   NaN             1  

In [15]:
# update 'conglomerate_test' to 25 if that value is null
site_df['conglomerate_test'] = np.where(site_df['conglomerate_test'].isnull(), 25, site_df['conglomerate_test'])
site_df[:5]

bed_dip bed_dip_direction  \
site_name                             
1              10               135   
1              10               135   
1             NaN               NaN   
2              12               140   
2              12               140   

                                            citations conglomerate_test  \
site_name                                                                 
1                                          This study                25   
1                                          This study                25   
1          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
2                                          This study                25   
2                                          This study                25   

          description dir_dec dir_inc dir_k dir_n_samples dir_polarity  \
site_name                                                                
1                 NaN   293.1    34.5   517             5            n   
1                 NaN   289.8    43.6   517             5            n   
1          VGP:Site 1   289.8    43.6   NaN             1          NaN   
2                 NaN   290.6    31.9   243             5            n   
2                 NaN   285.7      42   243             5            n   

               ...      geologic_types        lat lithologies  \
site_name      ...                                              
1              ...           Lava Flow        NaN      Basalt   
1              ...           Lava Flow        NaN      Basalt   
1              ...                 NaN  48.626429         NaN   
2              ...           Lava Flow        NaN      Basalt   
2              ...           Lava Flow        NaN      Basalt   

                                           location_name         lon  \
site_name                                                              
1          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
1          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
1          Osler Volcanics, Nipigon Strait, Upper Normal  271.894142   
2          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   
2          Osler Volcanics, Nipigon Strait, Upper Normal         NaN   

                                         method_codes result_type  \
site_name                                                           
1                      FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
1                      FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
1                          DE-DI:FS-LOC-GOOGLE:LP-DC2           i   
2          DE-K:LP-DC2:FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
2                      FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   

               vgp_lat      vgp_lon vgp_n_samples  
site_name                                          
1                  NaN          NaN           NaN  
1                  NaN          NaN           NaN  
1          31.65155202  185.5745062             1  
2                  NaN          NaN           NaN  
2                  NaN          NaN           NaN  

[5 rows x 22 columns]

In [16]:
# initialize a new site with no values
col_labels = site_df.columns
blank_site = pd.Series({}, index=col_labels, name='blank_site')#.fillna('')
blank_site

bed_dip               NaN
bed_dip_direction     NaN
citations             NaN
conglomerate_test     NaN
description           NaN
dir_dec               NaN
dir_inc               NaN
dir_k                 NaN
dir_n_samples         NaN
dir_polarity          NaN
dir_tilt_correction   NaN
geologic_classes      NaN
geologic_types        NaN
lat                   NaN
lithologies           NaN
location_name         NaN
lon                   NaN
method_codes          NaN
result_type           NaN
vgp_lat               NaN
vgp_lon               NaN
vgp_n_samples         NaN
Name: blank_site, dtype: float64

In [17]:
site_df = site_df.append(blank_site)
site_df.tail()

bed_dip bed_dip_direction  \
site_name                              
29             NaN               NaN   
30               6               129   
30               6               129   
30             NaN               NaN   
blank_site     NaN               NaN   

                                             citations conglomerate_test  \
site_name                                                                  
29          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
30                                          This study                25   
30                                          This study                25   
30          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
blank_site                                         NaN               NaN   

            description dir_dec dir_inc dir_k dir_n_samples dir_polarity  \
site_name                                                                  
29          VGP:Site 29   129.1   -54.5   NaN             1          NaN   
30                  NaN    98.7   -75.8   267             5            r   
30                  NaN    80.5   -80.4   267             5            r   
30          VGP:Site 30    80.5   -80.4   NaN             1          NaN   
blank_site          NaN     NaN     NaN   NaN           NaN          NaN   

                ...      geologic_types        lat lithologies  \
site_name       ...                                              
29              ...                 NaN  48.679736         NaN   
30              ...           Lava Flow        NaN      Basalt   
30              ...           Lava Flow        NaN      Basalt   
30              ...                 NaN  48.593955         NaN   
blank_site      ...                 NaN        NaN         NaN   

                                              location_name         lon  \
site_name                                                                 
29          Osler Volcanics, Nipigon Strait, Lower Reversed  271.897319   
30          Osler Volcanics, Nipigon Strait, Lower Reversed         NaN   
30          Osler Volcanics, Nipigon Strait, Lower Reversed         NaN   
30          Osler Volcanics, Nipigon Strait, Lower Reversed  271.782033   
blank_site                                              NaN         NaN   

                                          method_codes result_type  \
site_name                                                            
29                          DE-DI:FS-LOC-GOOGLE:LP-DC2           i   
30          DE-K:LP-DC2:FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
30          DE-K:LP-DC2:FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
30                          DE-DI:FS-LOC-GOOGLE:LP-DC2           i   
blank_site                                         NaN         NaN   

                vgp_lat      vgp_lon vgp_n_samples  
site_name                                           
29          50.54048276  181.0980435             1  
30                  NaN          NaN           NaN  
30                  NaN          NaN           NaN  
30          42.49393304  246.4019953             1  
blank_site          NaN          NaN           NaN  

[5 rows x 22 columns]

In [18]:
# copy a site from the site DataFrame, 
#change a few values, 
#then add the new site and the blank site to the site DataFrame
new_site = site_df.ix[2]
new_site['bed_dip'] = "other"
new_site.name = 'new_site'
site_df = site_df.append(new_site)
site_df.tail()

bed_dip bed_dip_direction  \
site_name                              
30               6               129   
30               6               129   
30             NaN               NaN   
blank_site     NaN               NaN   
new_site     other               NaN   

                                             citations conglomerate_test  \
site_name                                                                  
30                                          This study                25   
30                                          This study                25   
30          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
blank_site                                         NaN               NaN   
new_site    Cox & Doell 1960:McElhinny & McFadden 2000                ND   

            description dir_dec dir_inc dir_k dir_n_samples dir_polarity  \
site_name                                                                  
30                  NaN    98.7   -75.8   267             5            r   
30                  NaN    80.5   -80.4   267             5            r   
30          VGP:Site 30    80.5   -80.4   NaN             1          NaN   
blank_site          NaN     NaN     NaN   NaN           NaN          NaN   
new_site     VGP:Site 1   289.8    43.6   NaN             1          NaN   

                ...      geologic_types        lat lithologies  \
site_name       ...                                              
30              ...           Lava Flow        NaN      Basalt   
30              ...           Lava Flow        NaN      Basalt   
30              ...                 NaN  48.593955         NaN   
blank_site      ...                 NaN        NaN         NaN   
new_site        ...                 NaN  48.626429         NaN   

                                              location_name         lon  \
site_name                                                                 
30          Osler Volcanics, Nipigon Strait, Lower Reversed         NaN   
30          Osler Volcanics, Nipigon Strait, Lower Reversed         NaN   
30          Osler Volcanics, Nipigon Strait, Lower Reversed  271.782033   
blank_site                                              NaN         NaN   
new_site      Osler Volcanics, Nipigon Strait, Upper Normal  271.894142   

                                          method_codes result_type  \
site_name                                                            
30          DE-K:LP-DC2:FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
30          DE-K:LP-DC2:FS-FD:FS-H:FS-LOC-GOOGLE:SO-SM         NaN   
30                          DE-DI:FS-LOC-GOOGLE:LP-DC2           i   
blank_site                                         NaN         NaN   
new_site                    DE-DI:FS-LOC-GOOGLE:LP-DC2           i   

                vgp_lat      vgp_lon vgp_n_samples  
site_name                                           
30                  NaN          NaN           NaN  
30                  NaN          NaN           NaN  
30          42.49393304  246.4019953             1  
blank_site          NaN          NaN           NaN  
new_site    31.65155202  185.5745062             1  

[5 rows x 22 columns]

In [19]:
# get location DataFrame

def get_location():
    loc_data, dtype = pmag.magic_read(os.path.join(working_dir, 'locations.txt'))
    loc_df = DataFrame(loc_data)
    loc_df.index = loc_df['location_name']
    loc_df[loc_df == ''] = np.nan
    return loc_df
loc_df = get_location()
loc_df

citations  \
location_name                                                                                        
Osler Volcanics, Nipigon Strait, Lower Reversed                                         This study   
Osler Volcanics, Nipigon Strait, Upper Normal                                           This study   
Osler Volcanics, Nipigon Strait, Lower Reversed  Cox & Doell 1960:Fisher 1953:McElhinny & McFad...   
Osler Volcanics, Nipigon Strait, Upper Normal    Cox & Doell 1960:Fisher 1953:McElhinny & McFad...   

                                                conglomerate_test  \
location_name                                                       
Osler Volcanics, Nipigon Strait, Lower Reversed               NaN   
Osler Volcanics, Nipigon Strait, Upper Normal                 NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed                ND   
Osler Volcanics, Nipigon Strait, Upper Normal                  ND   

                                                contact_test continent_ocean  \
location_name                                                                  
Osler Volcanics, Nipigon Strait, Lower Reversed          NaN   North America   
Osler Volcanics, Nipigon Strait, Upper Normal            NaN   North America   
Osler Volcanics, Nipigon Strait, Lower Reversed           ND             NaN   
Osler Volcanics, Nipigon Strait, Upper Normal             ND             NaN   

                                                country         description  \
location_name                                                                 
Osler Volcanics, Nipigon Strait, Lower Reversed  Canada                 NaN   
Osler Volcanics, Nipigon Strait, Upper Normal    Canada                 NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed     NaN  All Reversed Flows   
Osler Volcanics, Nipigon Strait, Upper Normal       NaN    All Normal Flows   

                                                 dir_alpha95      dir_dec  \
location_name                                                               
Osler Volcanics, Nipigon Strait, Lower Reversed          NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal            NaN          NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed  4.624479368   114.967445   
Osler Volcanics, Nipigon Strait, Upper Normal     8.47928537  296.4616437   

                                                      dir_inc        dir_k  \
location_name                                                                
Osler Volcanics, Nipigon Strait, Lower Reversed           NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal             NaN          NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed  -57.57295621  40.16441995   
Osler Volcanics, Nipigon Strait, Upper Normal     39.46409514  82.38571031   

                                                       ...         \
location_name                                          ...          
Osler Volcanics, Nipigon Strait, Lower Reversed        ...          
Osler Volcanics, Nipigon Strait, Upper Normal          ...          
Osler Volcanics, Nipigon Strait, Lower Reversed        ...          
Osler Volcanics, Nipigon Strait, Upper Normal          ...          

                                                 pole_comp_name     pole_lat  \
location_name                                                                  
Osler Volcanics, Nipigon Strait, Lower Reversed             NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal               NaN          NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed  Characteristic  33.85573654   
Osler Volcanics, Nipigon Strait, Upper Normal    Characteristic   43.6574194   

                                                    pole_lon pole_n_sites  \
location_name                                                               
Osler Volcanics, Nipigon Strait, Lower Reversed          NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal 

In [20]:
# get list of all sites with the same 'location_name'
name = loc_df.ix[1].name
site_df[site_df['location_name'] == name].index

Index([u'1', u'1', u'1', u'2', u'2', u'2', u'3', u'3', u'3', u'4', u'4', u'4',
       u'5', u'5', u'5', u'new_site'],
      dtype='object', name=u'site_name')

In [21]:
# get all sites belonging to a particular location RECORD (i.e., what used to be a result)
# (NOT all sites with the same location name)
name = loc_df.ix[1].name
# not sure how to deal with duplicates...
loc_record = loc_df.ix[name].ix[1]
site_names = loc_record['site_names']
site_names = site_names.split(":")
# fancy indexing
site_df.ix[site_names]



bed_dip bed_dip_direction  \
site_name                             
1              10               135   
1              10               135   
1           other               NaN   
2              12               140   
2              12               140   
2             NaN               NaN   
3              14               146   
3              14               146   
3             NaN               NaN   
4              12               135   
4              12               135   
4             NaN               NaN   
5              10               135   
5              10               135   
5             NaN               NaN   

                                            citations conglomerate_test  \
site_name                                                                 
1                                          This study                25   
1                                          This study                25   
1          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
2                                          This study                25   
2                                          This study                25   
2          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
3                                          This study                25   
3                                          This study                25   
3          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
4                                          This study                25   
4                                          This study                25   
4          Cox & Doell 1960:McElhinny & McFadden 2000                ND   
5                                          This study                25   
5                                          This study                25   
5          Cox & Doell 1960:McElhinny & McFadden 2000                ND   

          description dir_dec dir_inc dir_k dir_n_samples dir_polarity  \
site_name                                                                
1                 NaN   293.1    34.5   517             5            n   
1                 NaN   289.8    43.6   517             5            n   
1          VGP:Site 1   289.8    43.6   NaN             1          NaN   
2                 NaN   290.6    31.9   243             5            n   
2                 NaN   285.7      42   243             5            n   
2          VGP:Site 2   285.7      42   NaN             1          NaN   
3                 NaN     310    24.6  2485             5            n   
3                 NaN   307.4    37.9  2485             5            n   
3          VGP:Site 3   307.4    37.9   NaN             1          NaN   
4                 NaN   303.9    18.2   331             5            n   
4                 NaN   302.8    29.9   331             5            n   
4          VGP:Site 4   302.8    29.9   NaN             1          NaN   
5                 NaN   297.3    33.1   130             5            n   
5                 NaN   294.7    42.5   130             5            n   
5          VGP:Site 5   294.7    42.5   NaN             1          NaN   

               ...      geologic_types        lat lithologies  \
site_name      ...                                              
1              ...           Lava Flow        NaN      Basalt   
1              ...           Lava Flow        NaN      Basalt   
1              ...                 NaN  48.626429         NaN   
2              ...           Lava Flow        NaN      Basalt   
2              ...           Lava Flow        NaN      Basalt   
2              ...                 NaN  48.637185         NaN   
3              ...           Lava Flow        NaN      Basalt   
3              ...           Lava Flow        NaN      Basalt   
3              ...                 NaN  48.640811         NaN   
4              ...           Lava Flow        NaN      Basalt   
4              ...           La

In [22]:
# rename a site
# first rename the index on the site_df
site_name = '1'
new_name = '111'
site_df.rename(index={site_name: new_name}, inplace=True)
# now, we need to fix the name in the location + sample tables

loc_df = get_location() # reset location

#loc_df.site_names.isin(['1:2:3:4:5'])

# some helper methods:
def split_if_str(x):
    if isinstance(x, str):
        return x.split(':')
    else:
        return x
    
def put_together_if_str(x):
    try:
        return ":".join(x)
    except TypeError:
        return x
    
def replace_colon_delimited_value(df, col_name, old_value, new_value):
    """
    Col must contain list
    """
    for index, row in df[df[col_name].notnull()].iterrows():
        names_list = row[col_name]
        try:
            ind = names_list.index(old_value)
        except ValueError:
            continue
        names_list[ind] = new_value

# split colon-delimited string into list, make new column of site_names as list, not string
loc_df['site_names_list'] = loc_df['site_names'].apply(split_if_str)         

# replace old value with new wherever it occurs in site_names_list
replace_colon_delimited_value(loc_df, "site_names_list", site_name, new_name)

# put list back into string form and update site_names
loc_df['site_names'] = loc_df['site_names_list'].apply(put_together_if_str)

# get rid of temporary column site_names_list
del loc_df['site_names_list']

# ta-da
loc_df

# would also need to change names in sample table, age table, and image table

citations  \
location_name                                                                                        
Osler Volcanics, Nipigon Strait, Lower Reversed                                         This study   
Osler Volcanics, Nipigon Strait, Upper Normal                                           This study   
Osler Volcanics, Nipigon Strait, Lower Reversed  Cox & Doell 1960:Fisher 1953:McElhinny & McFad...   
Osler Volcanics, Nipigon Strait, Upper Normal    Cox & Doell 1960:Fisher 1953:McElhinny & McFad...   

                                                conglomerate_test  \
location_name                                                       
Osler Volcanics, Nipigon Strait, Lower Reversed               NaN   
Osler Volcanics, Nipigon Strait, Upper Normal                 NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed                ND   
Osler Volcanics, Nipigon Strait, Upper Normal                  ND   

                                                contact_test continent_ocean  \
location_name                                                                  
Osler Volcanics, Nipigon Strait, Lower Reversed          NaN   North America   
Osler Volcanics, Nipigon Strait, Upper Normal            NaN   North America   
Osler Volcanics, Nipigon Strait, Lower Reversed           ND             NaN   
Osler Volcanics, Nipigon Strait, Upper Normal             ND             NaN   

                                                country         description  \
location_name                                                                 
Osler Volcanics, Nipigon Strait, Lower Reversed  Canada                 NaN   
Osler Volcanics, Nipigon Strait, Upper Normal    Canada                 NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed     NaN  All Reversed Flows   
Osler Volcanics, Nipigon Strait, Upper Normal       NaN    All Normal Flows   

                                                 dir_alpha95      dir_dec  \
location_name                                                               
Osler Volcanics, Nipigon Strait, Lower Reversed          NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal            NaN          NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed  4.624479368   114.967445   
Osler Volcanics, Nipigon Strait, Upper Normal     8.47928537  296.4616437   

                                                      dir_inc        dir_k  \
location_name                                                                
Osler Volcanics, Nipigon Strait, Lower Reversed           NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal             NaN          NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed  -57.57295621  40.16441995   
Osler Volcanics, Nipigon Strait, Upper Normal     39.46409514  82.38571031   

                                                       ...         \
location_name                                          ...          
Osler Volcanics, Nipigon Strait, Lower Reversed        ...          
Osler Volcanics, Nipigon Strait, Upper Normal          ...          
Osler Volcanics, Nipigon Strait, Lower Reversed        ...          
Osler Volcanics, Nipigon Strait, Upper Normal          ...          

                                                 pole_comp_name     pole_lat  \
location_name                                                                  
Osler Volcanics, Nipigon Strait, Lower Reversed             NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal               NaN          NaN   
Osler Volcanics, Nipigon Strait, Lower Reversed  Characteristic  33.85573654   
Osler Volcanics, Nipigon Strait, Upper Normal    Characteristic   43.6574194   

                                                    pole_lon pole_n_sites  \
location_name                                                               
Osler Volcanics, Nipigon Strait, Lower Reversed          NaN          NaN   
Osler Volcanics, Nipigon Strait, Upper Normal 

In [23]:
# merge location and site DataFrames
# basically will be site_df but with all of the relevant location
pd.merge(site_df, loc_df, 'inner', right_index=True, left_on=['location_name'])

location_name bed_dip  \
site_name                                                            
111          Osler Volcanics, Nipigon Strait, Upper Normal      10   
111          Osler Volcanics, Nipigon Strait, Upper Normal      10   
111          Osler Volcanics, Nipigon Strait, Upper Normal      10   
111          Osler Volcanics, Nipigon Strait, Upper Normal      10   
111          Osler Volcanics, Nipigon Strait, Upper Normal   other   
111          Osler Volcanics, Nipigon Strait, Upper Normal   other   
2            Osler Volcanics, Nipigon Strait, Upper Normal      12   
2            Osler Volcanics, Nipigon Strait, Upper Normal      12   
2            Osler Volcanics, Nipigon Strait, Upper Normal      12   
2            Osler Volcanics, Nipigon Strait, Upper Normal      12   
2            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
2            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
3            Osler Volcanics, Nipigon Strait, Upper Normal      14   
3            Osler Volcanics, Nipigon Strait, Upper Normal      14   
3            Osler Volcanics, Nipigon Strait, Upper Normal      14   
3            Osler Volcanics, Nipigon Strait, Upper Normal      14   
3            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
3            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
4            Osler Volcanics, Nipigon Strait, Upper Normal      12   
4            Osler Volcanics, Nipigon Strait, Upper Normal      12   
4            Osler Volcanics, Nipigon Strait, Upper Normal      12   
4            Osler Volcanics, Nipigon Strait, Upper Normal      12   
4            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
4            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
5            Osler Volcanics, Nipigon Strait, Upper Normal      10   
5            Osler Volcanics, Nipigon Strait, Upper Normal      10   
5            Osler Volcanics, Nipigon Strait, Upper Normal      10   
5            Osler Volcanics, Nipigon Strait, Upper Normal      10   
5            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
5            Osler Volcanics, Nipigon Strait, Upper Normal     NaN   
...                                                    ...     ...   
26         Osler Volcanics, Nipigon Strait, Lower Reversed      20   
26         Osler Volcanics, Nipigon Strait, Lower Reversed      20   
26         Osler Volcanics, Nipigon Strait, Lower Reversed      20   
26         Osler Volcanics, Nipigon Strait, Lower Reversed      20   
26         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
26         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
27         Osler Volcanics, Nipigon Strait, Lower Reversed      10   
27         Osler Volcanics, Nipigon Strait, Lower Reversed      10   
27         Osler Volcanics, Nipigon Strait, Lower Reversed      10   
27         Osler Volcanics, Nipigon Strait, Lower Reversed      10   
27         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
27         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
28         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
28         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
28         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
28         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
28         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
28         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
29         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
29         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
29         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
29         Osler Volcanics, Nipigon Strait, Lower Reversed       7   
29         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
29         Osler Volcanics, Nipigon Strait, Lower Reversed     NaN   
30         Osler Volcanics, Nipigon Strait, Lower Rever

## Pandas indexing

In [24]:
# first site
print site_df.ix[0][:5]
print '-'
# find site by index value
print site_df.ix['new_site'][:5]
print '-'
# return all sites' values for a col
print site_df['bed_dip'][:5]

bed_dip                      10
bed_dip_direction           135
citations            This study
conglomerate_test            25
description                 NaN
Name: 111, dtype: object
-
bed_dip                                                   other
bed_dip_direction                                           NaN
citations            Cox & Doell 1960:McElhinny & McFadden 2000
conglomerate_test                                            ND
description                                          VGP:Site 1
Name: new_site, dtype: object
-
site_name
111       10
111       10
111    other
2         12
2         12
Name: bed_dip, dtype: object
